In [ ]:
# loop = 0
# spot = 2
# base_off = 29
# m4f = False
# fisher = True
# # adv = False
# sample_size = 10
# lp_fct = 0

In [ ]:
BUILDPLATFORM = 'cw308t-stm32f415'
lvl = "l1"
%run "Setup_Disco_or_CWLITE.ipynb"
if "m4f" not in dir():
    global m4f
    m4f = False # implementation switch, either use m4f or mupq/opt implementation
if "adv" not in dir():
    global adv
    adv = False # use firmware with advanced trigger high routine
if "loop" not in dir():
    global loop
    loop = 0 # values {0,1,2}
if "fisher" not in dir():
    global fisher
    fisher = False

ff = f"{'' if m4f else 'mupq_'}crypto_kem_bikel1_{'fisher_yates' if fisher else 'set_bits'}{'_adv' if adv else ''}_{loop}_{'m4f' if m4f else 'opt'}_fi"

global lp_cy
if fisher:
    lp_cy = 130 if loop==0 else 53 if loop==1 else 28 if loop==2 else 0
else:
 lp_cy = 63 if loop==0 else 61 if loop==1 else 38 if loop==2 else 0
if adv: lp_cy = lp_cy + 6

import kat_bike as kat
import bike_key as bk

In [ ]:
if "flash" not in dir():
    global flash
    flash = True
if "lp_fct" not in dir():
    global lp_fct
    lp_fct = 0
if "sample_size" not in dir():
    global sample_size
    sample_size = 2
if "step" not in dir():
    global step
    step = lp_cy

if "base_off" in dir():
    global e_off_up
    global e_off_low
    e_off_up = base_off+lp_fct*lp_cy
    e_off_low = base_off

wait = "glitch_h0" in dir() and "glitch_h1" in dir()

def reboot_t(wait):
    reboot_flush()
    if wait:
        time.sleep(1.8)
        t_com.trig_h0(glitch_h0)
        t_com.trig_h1(glitch_h1)

In [ ]:
if flash:
    %run "Flash_Disco_CWLITE.ipynb"
reboot_t(wait)

In [ ]:
# show clock cycles with a high trigger pin
# run before setting up glitch output
import time
N = 1

scope.glitch.clk_src = 'clkgen'
scope.clock.adc_src = 'clkgen_x1' # default is 'clkgen_x4'
scope.glitch_disable()

reboot_t(wait)
start = time.time()

for _ in range(N):
    scope.arm()
    t_com.keygen()
    scope.capture()

    
dur = (time.time() - start) / N
st = dur*1.99

print(dur, st)

trigger_cnt = scope.adc.trig_count
print(trigger_cnt)

In [ ]:
aim = "clock"

scope.glitch.clk_src = 'clkgen'
scope.clock.adc_src = 'clkgen_x1'

# clock
scope.io.hs2 = "glitch"
scope.glitch.output = "clock_xor"

def set_width_offset(spot):
	if spot == 1:
		gc.set_range("width", -12, -12)
		gc.set_range("offset", -45,-45)
	elif spot == 2:
		gc.set_range("width", 10.1,11)
		gc.set_range("offset", -7,-7)
	elif spot == 3:
		gc.set_range("width", -15, -15)
		gc.set_range("offset", -49, -49)
	elif spot == 4:
		gc.set_range("width", 13, 13)
		gc.set_range("offset", 1, 1)
	elif spot == 5:
		gc.set_range("width", -10, -10)
		gc.set_range("offset", 45, 45)
	elif spot == 6:
		gc.set_range("width", -14, -14)
		gc.set_range("offset", 11, 11)
	elif spot == 7:
		gc.set_range("width", 23, 23)
		gc.set_range("offset", -26, -26)
	elif spot == 8:
		gc.set_range("width", -3, -3)
		gc.set_range("offset", 7, 7)
	elif spot == 9:
		gc.set_range("width", 47, 47)
		gc.set_range("offset", -46, -46)
	elif spot == 0:
		gc.set_range("width", 13, 13)
		gc.set_range("offset", -49, -49)
	
	else: raise ValueError(f"spot {spot} not valid.")

In [ ]:
# avoid glitching too close to triggers/aka added code
def skip_e_off(eo, loop, loop_cycles, fisher):
    bo = eo%loop_cycles
    if fisher:
        if loop==None: return False
        elif loop==0:
            if bo>122 or bo<4 or bo>95 and bo<112: 
                return True
        elif loop==1:
            if bo>43 or bo<4 or bo>14 and bo<31:
                return True
        elif loop==2:
            if eo<4 or eo>15:
                return True
        else: return False
    else:
        if loop==None: return False
        elif loop==0:
            if bo>52 or bo<4 or bo>26 and bo<43: 
                return True
        elif loop==1:
            if bo>50 or bo<4 or bo>24 and bo<41:
                return True
        elif loop==2:
            if eo>36 or eo<4 or eo>13 and eo<30:
                return True
        else: return False

def get_glitch_settings(scope):
    w=scope.glitch.width
    o=scope.glitch.offset
    e=scope.glitch.ext_offset
    r=scope.glitch.repeat
    src=scope.glitch.trigger_src
    return (w,o,e,r,src)

In [ ]:
key_dump = list()

In [ ]:
import matplotlib.pylab as plt
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

In [ ]:
import time

if not 'st' in dir():
    raise ValueError("sleep time 'st' not defined, run cell above.")

if "repeat" in dir():
    scope.glitch.repeat = repeat
else:
    scope.glitch.repeat = 1

if "e_off_low" in dir() and "e_off_up" in dir():
    gc.set_range("ext_offset", e_off_low, e_off_up)
else:
    gc.set_range("ext_offset", 2, lp_cy)

if "trig_src" in dir():
    scope.glitch.trigger_src = trig_src
else:
    scope.glitch.trigger_src = "ext_continuous"

gc.set_global_step(step)

keys = list()
reboot_t(wait)

tmp_spot = 0
tmp_spot_h = 10
if "spot" in dir():
    tmp_spot = spot
    tmp_spot_h = spot +1

if "w" in dir() and "o" in dir():
    tmp_spot = 0
    tmp_spot_h = 1    


for sp in range(10)[tmp_spot:tmp_spot_h]:
    if "w" in dir() and "o" in dir():
        gc.set_range("width", w, w)
        gc.set_range("offset", o, o)
    else:
        # width and offset spot
        set_width_offset(sp)
        print(f"start with spot {sp}")
    for glitch_settings in gc.glitch_values():
        eo = glitch_settings[2]
        if skip_e_off(eo, loop, lp_cy, fisher): continue
        scope.glitch.width = glitch_settings[0]
        scope.glitch.offset = glitch_settings[1]
        scope.glitch.ext_offset = glitch_settings[2]

        for _ in range(sample_size):
            scope.arm()
            t_com.keygen_async()
            time.sleep(st)
            
            ret = scope.capture()
            done = t_com.check_done_to()
            adc_state = scope.adc.state
            if ret or not done or adc_state:
                print('reset', end='')
                if ret: print(" - No Trigger seen", end='')
                if not done: 
                    print(" - Response not received in time", end='')
                    st = st * 1.2
                if adc_state: print(" - Trigger still high", end='')
                print()
                gc.add("reset")
                reboot_t(wait)
                continue

            key = bk.BIKE_key(t_com.r_sk_mupq(),lvl=t_com.lvl.name, mupq=True)
            result = bk.analyze_key(key.mupq_key, t_com.lvl)
            wl_ok = result[1][0] == t_com.lvl.d and result[1][1] == t_com.lvl.d
            sk_ok = result[2][0] == t_com.lvl.d and result[2][1] == t_com.lvl.d
            if fisher:
                # check if indices in wl are lower than D or higher than r
                ints0,ints1 = key.wlists_as_int
                for i in ints0:
                    if not wl_ok: break
                    wl_ok &= i>t_com.lvl.d and i<t_com.lvl.r_bits
                for i in ints1:
                    if not wl_ok: break
                    wl_ok &= i>t_com.lvl.d and i<t_com.lvl.r_bits
            cluster = False
            if not (wl_ok and sk_ok):
                h0_ints,h1_ints = bk.__get_sk_ilist(key)
                w0_ints,w1_ints = key.wlists_as_int
                max = 0
                for l in [h0_ints,h1_ints,w0_ints,w1_ints]:
                    tmp = bk.find_cluster(l, key.level.d, 5)[0][1]
                    max = tmp if tmp > max else max
                cluster = max > 8

            if wl_ok and sk_ok and not result[0].WK == bk.WL_Kind.MISMATCH and not cluster:
                gc.add("normal")
            else:
                print(f"\nkey {len(keys)}:")
                gc.add("success")
                print(f"Width: {scope.glitch.width}, Offset: {scope.glitch.offset}, External Offset: {scope.glitch.ext_offset}")
                keys.append((get_glitch_settings(scope), key))
                print(f"{result[0]}\nweight list {result[1]}, secret key {result[2]}")

    key_dump.extend(keys)
    keys = list()

In [ ]:
for k in key_dump:
    key = k[1]
    key.pk = bk.calculate_pk_from_sk(key.sk, kat.get_lvl(lvl))

In [ ]:
# save faulted keys
import shelve
with shelve.open("../traces/key_dump", writeback = True) as data:
	data["current"] = key_dump

In [ ]:
# save faulted keys
import shelve
with shelve.open("../traces/key_dump", writeback = True) as data:
	if ff in data.keys():
		data[ff].extend(key_dump)
	else: data[ff] = key_dump
key_dump = list()

In [ ]:
target.dis()
scope.dis()